In [50]:
!pip install census
!pip install us
!pip install addfips

In [21]:
from census import Census
from us import states
import API_KEYS
import pandas as pd

c = Census(API_KEYS.API_KEY)

In [49]:

for i in range(2009, 2023):
    if i == 2009:
        data = c.acs5dp.state_county(('DP03_0006E', 'DP03_0009PE', 'DP03_0064E', 'DP03_0063E', 'DP04_0003E', 'DP04_0005E', 'DP03_0103E', 'DP04_0088E','NAME'), '*', '*', year = i) 
        #mil pop, unemployment %, mean income, median income, vacant housing units, rental vacany rate, % below poverty, median house value
        #https://api.census.gov/data/2009/acs/acs5/profile/variables/DP03_0006E.html
    if i >= 2015:
        data = c.acs5dp.state_county(('DP03_0006E', 'DP03_0009PE', 'DP03_0063E', 'DP03_0062E', 'DP04_0003E', 'DP04_0005E', 'DP03_0119PE', 'DP04_0089E','NAME'), '*', '*', year = i)
        #mil pop, unemployment %, mean income, median income, vacant housing units, rental vacany rate, % below poverty, median house value
        #https://api.census.gov/data/2022/acs/acs5/profile/variables.html
    else:
        data = c.acs5dp.state_county(('DP03_0006E', 'DP03_0009PE', 'DP03_0063E', 'DP03_0062E', 'DP04_0003E', 'DP04_0005E', 'DP03_0119PE', 'DP04_0088E','NAME'), '*', '*', year = i) 
        #mil pop, unemployment %, mean income, median income, vacant housing units, rental vacany rate, % below poverty, median house value
        #https://api.census.gov/data/2010/acs/acs5/profile/variables.html
    
    cen_data = pd.DataFrame(data)
    cen_data[['county_name', 'state_name']] = cen_data['NAME'].str.split(',', expand = True)
    cen_data.columns = ['mil pop', 'unemployment %', 'mean income', 'median income', 'vacant housing units', 'rental vacany rate', '% below poverty line', 'median house value', *cen_data.columns[8:]]
    cen_data.to_csv(str(i)+'_census_data.csv', index = False)
    print(i)

2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022


In [ ]:
#https://api.census.gov/data/1992/cps/asec/mar/variables.html
#P_STAT(2), 
#mil pop, unemployment %, mean income, median income, vacant housing units, rental vacany rate, % below poverty, median house value

In [ ]:
#https://api.census.gov/data/1994/cps/basic/jan/variables.html
#PEAFNOW, PREXPLF (2/(1+2)), HUFAMINC (map to income levels), 
#mil pop, unemployment %, mean income, 

In [62]:
#median house value
df_house = pd.read_excel('hpi_at_bdl_county.xlsx', skiprows = 6) #https://www.fhfa.gov/data/hpi/datasets?tab=additional-data

In [63]:
df_house

,State,County,FIPS code,Year,Annual Change (%),HPI,HPI with 1990 base,HPI with 2000 base
0,AL,Autauga,1001,1986,.,100.00,95.59,70.96
1,AL,Autauga,1001,1987,-1.94,98.06,93.74,69.59
2,AL,Autauga,1001,1988,2.57,100.58,96.14,71.38
3,AL,Autauga,1001,1989,4.32,104.92,100.29,74.46
4,AL,Autauga,1001,1990,-0.29,104.62,100.00,74.24
...,...,...,...,...,...,...,...,...
100427,WY,Weston,56045,2019,8.33,223.40,.,201.09
100428,WY,Weston,56045,2020,4.36,233.13,.,209.86
100429,WY,Weston,56045,2021,4.93,244.62,.,220.20
100430,WY,Weston,56045,2022,6.87,261.41,.,235.32


In [ ]:
#TO FIND PRE 2009
#median income, vacant housing units, rental vacany rate, % below poverty, 